In [13]:
# importing necessary modules.
import pandas as pd
import config as cfg
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

In [14]:
# creating a connection to an aws rds postgres cloud database.
rds_connection_string = f'postgres:{cfg.password}@test-db.cy2enoewwvsi.us-east-2.rds.amazonaws.com:5432/stocks_db'
engine = create_engine(f'postgres://{rds_connection_string}')

In [15]:
# checking out the table names in the database.
engine.table_names()

['russell_2000', 'sandp', 'sandp_russell']

In [16]:
# checking out the SandP_500 table.
sandp_russell = pd.read_sql_query('SELECT * FROM sandp_russell', con=engine)
sandp_russell.head()

,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,enterprise_value_multiple,gross_profit_margin,pretax_profit_margin,net_profit_margin,tax_rate,return_on_assets,return_on_equity,return_on_capital_employed,receivables_turnover_1,payables_turnover,inventory_turnover,fixed_asset_turnover,asset_turnover,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_inventory_outstanding,operating_cycle,days_of_payables_outstanding,cash_conversion_cycle,debt_ratio,debt_to_equity_ratio,longterm_debt_to_capitalization,total_debt_to_capitalization,interest_coverage,cash_flow_to_debt,company_equity_multiplier,operating_cash_flow_per_share,free_cash_flow_per_share,cash_per_share,payout_ratio,receivables_turnover_2,operating_cash_flow_to_sales,free_cash_flow_to_operating_cash_flow,cash_flow_coverage_ratio,short_term_coverage_ratio,capital_expenditure_coverage_ratio,dividend_capex_coverage_ratio,dividend_payout_ratio,gross_profit_growth,ebit_growth,operating_income_growth,net_income_growth,eps_growth,eps_diluted_growth,weighted_average_share_growth,weighted_averages_shares_diluted_growth,dividents_per_share_growth,operating_cash_flow_growth,free_cash_flow_growth,ten_year_revenue_growth_per_share,five_year_revenue_growth_per_share,three_year_revenue_growth_per_share,ten_year_operating_cf_growth_per_share,five_year_operating_cf_growth_per_share,three_year_operating_cf_growth_per_share,ten_year_net_income_growth_per_share,five_year_net_income_growth_per_share,three_year_net_income_growth_per_share,ten_year_shareholders_equity_growth_per_share,five_year_shareholders_equity_growth_per_share,three_year_shareholders_equity_growth_per_share,ten_year_dividend_growth_per_share,five_year_dividend_growth_per_share,three_year_dividend_growth_per_share,receivables_growh,inventory_growth,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment,above_below_sandp_return,market_cap,industry,sector
0,A,"Agilent Technologies, Inc.",2.5517,5.17165,1.48130,6.30885,7.0014,18.18070,13.57995,16.746223,0.524850,0.057228,0.059362,2.720066,0.0799,0.09975,0.0513,7.0014,3.85290,8.0338,5.429030,0.575072,2.7640,2.894055,2.481972,-97.99795,53.36445,-99.0,29.23140,-99.0,0.38115,1.15130,0.470496,0.535164,4.6439,0.167487,3.020614,1.62420,1.2763,11.89585,-99.0000,7.0014,0.111470,0.764976,0.167487,204.239174,4.755422,4.755422,-99.000000,-0.00225,3.19800,5.05085,11.00990,10.92090,10.75375,-0.02195,-0.0236,-99.00000,0.14975,0.30285,-0.03010,0.08565,0.04200,0.04210,-0.00300,-0.06355,0.0079,0.12195,0.03385,-0.02165,0.01000,-0.00400,-99.00000,-99.00000,-99.0000,0.11660,0.07580,0.18005,0.15185,0.31885,-0.0674,0.01880,215.407785,1,7.462700e+09,Medical Diagnostics & Research,Healthcare
1,AAL,"American Airlines Group, Inc.",-99.0000,-99.00000,0.11565,-99.00000,27.3348,-99.00000,2.20925,61.812855,0.495744,-0.018258,-0.047475,-99.000000,-0.0689,0.27010,-0.0152,27.3348,5.02710,37.6691,1.378462,0.833326,0.8190,0.687756,0.074855,-19.81890,13.11230,-99.0,19.26545,-99.0,0.45185,-3.08765,1.617521,1.486838,-0.4848,0.095474,-99.000000,3.26045,-1.8500,1.84910,-99.0000,27.3348,0.051335,-0.565762,0.095474,0.746942,0.638704,0.638704,-99.000000,0.03935,0.77400,0.88765,0.49305,0.55295,0.55295,0.09755,-99.0000,-99.00000,1.00075,0.16005,-0.06610,-0.12145,-0.13695,-0.15725,-0.09305,-0.27055,-99.0000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.0000,-0.04605,0.06370,-0.00170,-0.06235,0.00975,-99.0000,0.01435,492.976589,1,6.133809e+08,Airlines,Industrials
2,AAN,"Aaron's, Inc.",1.5315,1.95610,0.78855,12.37860,27.7757,59.12785,19.92670,0.126805,0.812846,0.103087,0.063665,0.370173,-99.0000,0.12390,-99.0000,27.7757,2.51545,-99.0000,1.896709,1.287786,-99.0000,-99.000000,-99.000000,-99.00000,13.65740,-99.0,39.02640,-99.0,0.03475,0.05235,-99.000000,-99.000000,52.3004,-99.000000,1.511503,1.49430,1.0025,1.1184

In [17]:
industry = pd.get_dummies(sandp_russell['industry'])

In [18]:
sector = pd.get_dummies(sandp_russell['sector'])

In [19]:
sandp_russell = pd.concat([sandp_russell, industry, sector], axis=1)

In [20]:
sandp_russell.drop(['industry', 'sector'], axis=1, inplace=True)

In [21]:
# dropping unnecessary columns.
sandp_russell.drop(['symbol', 'name', 'percent_return_on_investment'], axis=1, inplace=True)

In [23]:
X = sandp_russell.drop('above_below_sandp_return', axis=1)
target = sandp_russell['above_below_sandp_return']

In [24]:
names = []
for x in X.columns:
    names.append(x)

In [25]:
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X, target)
rf.score(X, target)

1.0

In [26]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([1.58712141e-02, 1.70084741e-02, 1.88910713e-02, 1.09779824e-02,
       1.05782607e-02, 1.29649563e-02, 1.46537562e-02, 1.35150047e-02,
       1.16649919e-02, 1.55959154e-02, 1.60744329e-02, 1.10114540e-02,
       1.17739533e-02, 1.46133822e-02, 9.87727793e-03, 9.97949500e-03,
       1.36880769e-02, 9.53773658e-03, 1.78336598e-02, 1.45079218e-02,
       1.11196810e-02, 1.10717618e-02, 1.01712395e-02, 8.87601530e-03,
       1.09114902e-02, 2.03400925e-03, 1.56470504e-02, 1.59588552e-03,
       1.26294412e-02, 1.19665132e-02, 7.59717436e-03, 8.15128195e-03,
       9.92590633e-03, 8.58245520e-03, 1.52221625e-02, 1.65060780e-02,
       1.54333770e-02, 1.76088203e-02, 9.32698629e-03, 1.07974637e-02,
       1.48258636e-02, 1.35729887e-02, 8.03287382e-03, 8.67218085e-03,
       1.49128026e-02, 1.94477837e-02, 8.42216055e-03, 1.39794932e-02,
       1.25478332e-02, 1.39574472e-02, 1.19399514e-02, 1.12216002e-02,
       1.20427515e-02, 1.13725249e-02, 1.14003429e-02, 6.63948200e-03,
      

In [27]:
# We can sort the features by their importance
features = sorted(zip(rf.feature_importances_, X.columns), reverse=True)

In [41]:
features

[(0.03176306991252214, 'market_cap'),
 (0.019447783658668612, 'dividend_capex_coverage_ratio'),
 (0.018891071279405564, 'price_to_sales'),
 (0.017833659787364106, 'fixed_asset_turnover'),
 (0.01760882029622336, 'cash_per_share'),
 (0.01700847412534801, 'price_to_book'),
 (0.016506077967158396, 'operating_cash_flow_per_share'),
 (0.016074432917614477, 'net_profit_margin'),
 (0.015871214056002086, 'price_to_bookvalue'),
 (0.01564705039339082, 'days_of_payables_outstanding'),
 (0.015595915434574215, 'pretax_profit_margin'),
 (0.01543337703328161, 'free_cash_flow_per_share'),
 (0.015243224713474177, 'operating_cash_flow_growth'),
 (0.015222162469798052, 'company_equity_multiplier'),
 (0.014912802602954982, 'capital_expenditure_coverage_ratio'),
 (0.014825863606381433, 'operating_cash_flow_to_sales'),
 (0.014653756177573394, 'price_to_operating_cash_flow'),
 (0.014613382214724642, 'return_on_equity'),
 (0.014507921750447104, 'asset_turnover'),
 (0.014268165050646063, 'asset_growth'),
 (0.01

In [30]:
features = features[0:90]

In [31]:
blank = []
for feature in features:
    blank.append(feature[1])

In [40]:
X = sandp_russell[blank]
target = sandp_russell['above_below_sandp_return']

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, target_train, target_test = train_test_split(X, target, random_state=1, stratify=target)

In [34]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
classifier.fit(X_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
print(f'Training Data Score: {classifier.score(X_train, target_train)}')
print(f'Testing Data Score: {classifier.score(X_test, target_test)}')

Training Data Score: 0.5813953488372093
Testing Data Score: 0.5823665893271461


In [37]:
predictions = classifier.predict(X_test)
pd.DataFrame({'Prediction': predictions, 'Actual': target_test}).reset_index(drop=True).head()

,Prediction,Actual
0,0,0
1,0,0
2,0,1
3,0,0
4,0,0


In [38]:
target_test.value_counts()
1 - target_test.mean()

0.5823665893271461